In [95]:

from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import BoxScore

def fetch_finished_games():
    games = scoreboard.ScoreBoard()
    games = games.get_dict()["scoreboard"]["games"]
    game_ids = [game['gameId'] for game in games if game['gameStatusText'] == 'Final' ]
    return game_ids

def fetch_game_results(game_ids):
    res = []
    for game_id in game_ids:
        box_score = BoxScore(game_id=game_id).get_dict()['game']
        away_team = box_score['awayTeam']
        home_team = box_score['homeTeam']

        winner = away_team  if away_team['score'] > home_team['score'] else home_team
        loser = away_team  if away_team['score'] < home_team['score'] else home_team

        res.append((winner, loser, box_score))

    return res

In [96]:
import shelve
import os
from pathlib import Path

def get_shelve_fp():
    dir = os.environ['NBPYTHONRUNNER_DATADIR'] if 'NBPYTHONRUNNER_DATADIR' in os.environ else  'C:\\Users\\DevelopPC\\Desktop'
    dir_p = Path(dir)
    fp = dir_p.joinpath("nba_games_processed.shelve")
    return fp

def process_game_id(game_id: str):
    with shelve.open(get_shelve_fp(), writeback=True) as shelve_dict:
        if "game_ids" not in shelve_dict:
            shelve_dict["game_ids"] = [game_id]
            return
        shelve_dict["game_ids"].append(game_id)

def is_game_processed(game_id: str)-> bool:
    with shelve.open(get_shelve_fp()) as shelve_dict:
        if "game_ids" not in shelve_dict:
            return False
        game_ids = shelve_dict["game_ids"]
        return game_id in game_ids

In [97]:
from munch import DefaultMunch
import pandas as pd

def generate_post_content(winner, loser, box_score):
    players = winner['players']
    best_players = list(filter(lambda a: a['statistics']['points'] > 30, players))
    game_id = box_score["gameId"]

    if not best_players or is_game_processed(game_id):
        print(f"Already processed or no 30 point performance for {game_id}")
        return (None, None, None)
    
    best_player = best_players[0]

    stats = DefaultMunch.fromDict(best_player['statistics'])

    title =     f"{best_player['firstName']} {best_player['familyName']} "
    title +=    f"scores {stats.points} pts, {stats.reboundsTotal} reb, and {stats.assists} ast "
    title +=    f"on {stats.fieldGoalsPercentage*100:0.0f}/{stats.threePointersPercentage*100:0.0f}/{stats.freeThrowsPercentage*100:0.0f} "
    title +=    f"shooting splits in the win against the {loser['teamName']}"

    df = pd.DataFrame([stats.toDict()])
    COLS = {
        "points": "PTS",
        "assists": "AST",
        "plusMinusPoints": "+/-",
        "foulsPersonal": "PF",
        "blocks":"BLK",
        "steals": "STL",
        "minutes": "MIN",
        "foulsDrawn": "Fouls Drawn",
    }
    df.rename(columns=COLS, inplace=True)
    selftext = df.iloc[:, df.columns.isin(COLS.values())].to_markdown(index="False")

    return (title, selftext, game_id)

In [98]:
import praw
import os

def create_score_post(title, selftext):
    reddit = praw.Reddit(
        client_id=os.environ["REDDIT_API_CLIENT_ID"],
        client_secret=os.environ["REDDIT_API_SECRET"],
        password=os.environ["REDDIT_API_PASSWORD"],
        user_agent=os.environ["REDDIT_API_USER_AGENT"],
        username=os.environ["REDDIT_API_USER"],
    )

    drafts = reddit.drafts()

    draft = reddit.drafts.create(title=title, selftext=selftext, subreddit=reddit.user.me().subreddit)
    draft.submit()

In [ ]:

games = fetch_finished_games()
results = fetch_game_results(games)

for result in results:
    if not result:
        continue
    winner, loser, box_score = result
    title, selftext, gameId = generate_post_content(winner, loser, box_score)
    if not title:
        continue
    create_score_post(title, selftext)
    print(f"Created post with {title} and {selftext}")
    process_game_id(gameId)
